<a href="https://colab.research.google.com/github/brendaar1991/BArellanoIntro/blob/main/Copy_of_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Define the discriminator model
def discriminator(in_shape = (28, 28, 1)):
    # Construct the model
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides = (2, 2), padding = 'same', input_shape = in_shape))
    model.add(LeakyReLU(negative_slope = 0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3, 3), strides = (2, 2), padding = 'same'))
    model.add(LeakyReLU(negative_slope = 0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))

    # Compile the model
    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])
    return model

In [ ]:
# Imports
import numpy as np
from numpy.random import randn, randint
import matplotlib.pyplot as plt
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout

In [ ]:
# Define the generator model
def generator(latent_dim):
    # Construct the model
    model = Sequential()

    # Foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim = latent_dim))
    model.add(LeakyReLU(negative_slope = 0.2))
    model.add(Reshape((7, 7, 128)))

    # Upsample to 14x14
    model.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same'))
    model.add(LeakyReLU(negative_slope = 0.2))

    # Upsample to 28x28
    model.add(Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same'))
    model.add(LeakyReLU(negative_slope = 0.2))
    model.add(Conv2D(1, (7, 7), activation = 'sigmoid', padding = 'same'))
    return model

In [ ]:
# Define the GAN (i.e., combine the generator and discriminator)
def GAN(g_model, d_model):
    # Make weights in the discriminator not trainable
    d_model.trainable = False

    # Construct the GAN model
    model = Sequential()
    model.add(g_model)
    model.add(d_model)

    # Compile the model
    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt)
    return model

In [ ]:
# Load and prepare images from MNIST dataset
def load_real_samples():
    # Load the dataset
    (trainX, _), (_, _) = load_data()

    # Expand the images to 3D
    X = np.expand_dims(trainX, axis = -1)

    # Convert images from unsigned ints to floats
    X = X.astype('float32')

    # Scale images from [0,255] t0 [0,1] (i.e., normalize)
    X = X / 255.0
    return X

In [ ]:
# Select real samples
def generate_real_samples(dataset, n_samples):
    # Choose random instances and retrieve selected images
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]

    # Generate 'real' class labels (i.e., 1)
    y = np.ones((n_samples, 1))
    return X, y

In [ ]:
# Generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # Generate points in the latent space
    x_input = randn(latent_dim * n_samples)

    # Reshape points into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [ ]:
# Generate fake samples with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # Generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)

    # Predict outputs
    X = g_model.predict(x_input)

    # Create 'fake' class labels (i.e., 0)
    y = np.zeros((n_samples, 1))
    return X, y

In [ ]:
# Create and save a plot of generated images
def save_plot(examples, epoch, n = 10):
    # Plot the images
    for i in range(n * n):
        # Define subplot and turn off axis
        plt.subplot(n, n, 1 + i)
        plt.axis('off')

        # Plot raw pixel data
        plt.imshow(examples[i, :, :, 0])

    # Save plot to file
    filename = f'generated_plot_e{epoch+1}.png'
    plt.savefig(filename)
    plt.close()

In [ ]:
# Evaluate the discriminator, plot the generated images, and save the generator
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples = 100):
    # Prepare real samples and evaluate the discriminator on them
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose = 0)

    # Prepare fake samples and evaluate the discriminator on them
    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose = 0)

    # Summarize the discriminator's performance
    f = open('summaries.txt', 'a')
    f.write(f'>Accuracy real: {acc_real*100:.0f}%, fake: {acc_fake*100:.0f}%\n')
    f.close()
    #print(f'>Accuracy real: {acc_real*100:.0f}%, fake: {acc_fake*100:.0f}%')

    # Save the plot and generator
    save_plot(X_fake, epoch)
    filename = f'generator_model_{epoch+1}.h5'
    g_model.save(filename)

In [ ]:
# Train the generator and discriminator
def train(g_model, d_model, GAN_model, dataset, latent_dim, n_epochs = 100, n_batch = 256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # Manually enumerate epochs
    for i in range(n_epochs):
        # Enumerate batches over the training set
        for j in range(bat_per_epo):
            # Get randomly selected 'real' samples and generate 'fake' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

            # Create training set for the discriminator
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))

            # Update the discriminator's weights
            d_loss, _ = d_model.train_on_batch(X, y)

            # Prepare points in the latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)

            # Create inverted labels for the fake samples (to fool the discriminator)
            y_gan = np.ones((n_batch, 1))

            # Update the generator via the discriminator's error
            g_loss = GAN_model.train_on_batch(X_gan, y_gan)

            # Summarize loss on this batch
            f = open('summaries.txt', 'a')
            f.write(f'>{i+1}, {j+1}/{bat_per_epo}, d = {d_loss:.3f}, g = {g_loss[-1]:.3f}\n')
            f.close()
            #print(f'>{i+1}, {j+1}/{bat_per_epo}, d = {d_loss:.3f}, g = {g_loss[-1]:.3f}')

        # Evaluate the model performance every 10 epochs
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
# Define size of the latent space
latent_dim = 100

# Create the discriminator and generator models
d_model = discriminator()
g_model = generator(latent_dim)

# Create the GAN model
GAN_model = GAN(g_model, d_model)

# Load image data
dataset = load_real_samples()

# Train the GAN model
train(g_model, d_model, GAN_model, dataset, latent_dim, n_epochs = 10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 241ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 255ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 251ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 253ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 